In [1]:
import pyspark
from pyspark.sql import functions as F
from pyspark.sql.functions import desc
from pyspark.sql.window import Window
import datetime
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("Alti bd2") \
                    .setMaster('local') \
                    .set('spark.executor.memory','5G') \
                    .set('spark.cores.max', '24') \
                    .set('spark.sql.tungsten.enabled', 'true')

sc = SparkContext(conf=conf)

sc._jsc.hadoopConfiguration().set('textinputformat.record.delimiter', '\r\n\r\n')

In [2]:
spark = pyspark.sql.SparkSession.builder \
            .master('local') \
            .appName('Altigran BD2') \
            .getOrCreate() 

file = './data/amazon-meta.txt'
data = sc.textFile(file)

In [3]:
spark

## Utils

In [4]:
def parse(text):
    lines = text.split('\r\n')
    obj = {}
    ID = -1
    reading_categories = False
    reading_reviews = False
    for i in lines:

        if reading_categories:
            categories = i.split('|')
            if(len(categories) == 1):
                reading_categories = False
            else:
                obj['CATEGORIES']['categories'].append(categories[len(categories)-1])

        if reading_reviews:
            reviewsObj = {}
            reviewsObj['date'] = i.split(' cutomer')[0].replace(' ', '')
            reviews = i.split(': ')
            reviewsObj['customer'] = reviews[1].split(' rating')[0].replace(' ', '')
            reviewsObj['rating'] = reviews[2].split(' ')[0].replace(' ', '')
            reviewsObj['votes'] = reviews[3].split(
                ' helpful')[0].replace(' ', '')
            reviewsObj['helpful'] = reviews[4].replace(' ', '')
            obj['REVIEWS']['reviews'].append(reviewsObj)

        elif 'Id:  ' in i:
            obj['ID'] = i.split('Id:')[1].replace(' ', '')
            ID = obj['ID']
        elif 'ASIN:' in i:
            obj['ASIN'] = i.split('ASIN:')[1].replace(' ', '')
        elif 'title:' in i:
            title = i.split('title: ')[1]
            obj['TITLE'] = title
        elif 'group:' in i:
            obj['GROUP'] = i.split('group: ')[1]
        elif 'salesrank:' in i:
            obj['SALESRANK'] = i.split('salesrank: ')[1]
        elif 'similar:' in i:
            similars = i.split(': ')[1].split('  ')
            number_of_similars = similars[0]
            similars.pop(0)
            obj['SIMILARS'] = {'size': number_of_similars, 'similars': similars}
        elif 'categories:' in i:
            number_of_categories = i.split(': ')[1]
            obj['CATEGORIES'] = {
                'size': number_of_categories, 'categories': []}
            reading_categories = True
        if 'reviews:' in i:
            reviews_split = i.split(': ')
            total = reviews_split[2].split('  ')[0]
            downloaded = reviews_split[3].split('  ')[0]
            avg_rating = reviews_split[4]
            obj['REVIEWS'] = {'total': total, 'downloaded': downloaded,
                            'avg_rating': avg_rating, 'reviews': []}
            reading_reviews = True
    return obj


# Doc = Text Document
def filterById(doc, selected_id):
    lines = doc.split('\n')
    for line in lines:
        if 'Id:  ' in line:
            ID = int(line.split('Id:')[1].replace(' ', ''))
            if( ID == selected_id ):
                return True
            else:
                return None

# Just to print
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

## 1. Create the dataframes using RDDs

Professor, eu tentei criar os DataFrames em uma única passada emitindo tuplas com IDs referenciando tabelas (como podes ver no código comentado), funcionou porém acabou com a minha memória.

Para que isso não ocorra no seu pc, fiz da forma B: cada passada nos arquivos constrói uma relação.

In [5]:
# TUPLE_PRODUCTS_ID = 0
# TUPLE_SIMILARS_ID = 1
# TUPLE_CATEGORIES_ID = 2
# TUPLE_REVIEWS_ID = 3

# KEY = 0
# VALUE = 1

# def emitProductsRelation(obj):
#     try:
#         return [ (TUPLE_PRODUCTS_ID, ( int(obj['ID']), int(obj['ASIN']), obj['TITLE'], obj['GROUP'], int(obj['SALESRANK']), \
#                int(obj['REVIEWS']['total']), int(obj['REVIEWS']['downloaded']), int(obj['REVIEWS']['avg_rating']) ) )]
#     except Exception as e:
#         return []
        
# def emitSimilarsRelation(obj):
#     try:
#         ID = int(obj['ID'])
#         array = [ ( TUPLE_SIMILARS_ID, ( ID, int(similar) ) ) for similar in obj['SIMILARS']['similars'] ]
#         return array
#     except Exception as e:
#         return []
    
# def emitCategoriesRelation(obj):
#     try:
#         ID = int(obj['ID'])
#         array = [ ( TUPLE_CATEGORIES_ID, ( ID, categories ) ) for categories in obj['CATEGORIES']['categories'] ]
#         return array
#     except Exception as e:
#         return []
    
# def emitReviewsRelation(obj):
#     try:
#         ID = int(obj['ID'])
#         array = [ (TUPLE_REVIEWS_ID, ( ID, r['date'], r['customer'], int(r['rating']), int(r['votes']), int(r['helpful']) ) ) \
#                  for r in obj['REVIEWS']['reviews'] ]
#         return array
#     except Exception as e:
#         return []

# # Get each data from the textual document in the following format:
# # ( TUPLE_ID, ARRAY )
# # With TUPLE_ID: PRODUCT_ID  ||  CATEGORY_ID  || REVIEW_ID
# def emitRelations(doc):
#     obj = parse(doc)
#     products = emitProductsRelation(obj)
#     similars = emitSimilarsRelation(obj)
#     categories = emitCategoriesRelation(obj)
#     reviews = emitReviewsRelation(obj)
#     return products + similars + categories + reviews

# rdd = data.flatMap( emitRelations ) \
#             .groupByKey() \
#             .map( lambda x: ( x[KEY], list(x[VALUEmini])))

# products_relation = rdd.filter( lambda x: x[0] == TUPLE_PRODUCTS_ID ).collect()[0][VALUE]
# similars_relation = rdd.filter( lambda x: x[0] == TUPLE_SIMILARS_ID ).collect()[0][VALUE]
# categories_relation = rdd.filter( lambda x: x[0] == TUPLE_CATEGORIES_ID ).collect()[0][VALUE]
# reviews_relation = rdd.filter( lambda x: x[0] == TUPLE_REVIEWS_ID ).collect()[0][VALUE]

# abc = rdd.collect()

In [6]:
def emitProductsRelation(doc):
    obj = parse(doc)
    try:
        return( int(obj['ID']), obj['ASIN'], obj['TITLE'], obj['GROUP'], int(obj['SALESRANK']), \
               int(obj['REVIEWS']['total']), int(obj['REVIEWS']['downloaded']), float(obj['REVIEWS']['avg_rating']) )
    except Exception as e:
        pass
    
rdd = data.map( emitProductsRelation ) \
          .filter(lambda x: x!=None)

productsDF = rdd.toDF(['id', 'asin', 'title', 'group', 'salesrank', 'r_total', 'r_downloaded', 'r_avg_rating' ])
productsDF.createOrReplaceTempView('products')

In [7]:
def emitSimilarsRelation(doc):
    obj = parse(doc)
    array = []
    try:
        ID = int(obj['ID'])
        array = [ ( ID, similar ) for similar in obj['SIMILARS']['similars'] ]
        return array
    except Exception as e:
        return array
    
rdd = data.flatMap( emitSimilarsRelation )

similarsDF = rdd.toDF(['productID', 'asin'])
similarsDF.createOrReplaceTempView('similars')

In [8]:
def emitCategoriesRelation(doc):
    obj = parse(doc)
    array = []
    try:
        ID = int(obj['ID'])
        array = [ ( ID, categories ) for categories in obj['CATEGORIES']['categories'] ]
        return array
    except Exception as e:
        return array
    
rdd = data.flatMap( emitCategoriesRelation )

categoriesDF = rdd.toDF(['productID', 'category'])
categoriesDF.createOrReplaceTempView('categories')

In [9]:
def emitReviewsRelation(doc):
    obj = parse(doc)
    array = []
    try:
        ID = int(obj['ID'])
        array = [ ( ID, datetime.datetime.strptime( r['date'], '%Y-%m-%d').date(), \
                   r['customer'], int(r['rating']), int(r['votes']), int(r['helpful']) ) \
                 for r in obj['REVIEWS']['reviews'] ]
        return array
    except Exception as e:
        return array

rdd = data.flatMap( emitReviewsRelation )

reviewsDF = rdd.toDF(['productID', 'date', 'customer', 'rating', 'votes', 'helpful'])
reviewsDF.createOrReplaceTempView('reviews')

---

## Questão A

### Dado um produto, listar:
### --- Os 5 comentários mais úteis e com a maior avaliação.
### --- Os 5 comentários mais úteis e com a menor avaliação.

## Usando SQL

Estratégia: Criar uma **VIEW** com os 5 comentários mais úteis.
- Ordenar view por rating de forma ascendente
- Ordenar view por rating de forma decrescente

In [10]:
SELECTED_ID = 2

most_useful_comments_view = spark.sql("""
    SELECT *
        FROM reviews r
        WHERE r.productID = {}
        SORT BY r.helpful DESC
        LIMIT 5
""".format(SELECTED_ID))
most_useful_comments_view.createOrReplaceTempView('most_useful_comments')

### [SQL] - Os 5 comentários mais úteis e com a MAIOR avaliação.

In [11]:
spark.sql('SELECT * FROM most_useful_comments ORDER BY rating DESC').show(1000)

+---------+----------+--------------+------+-----+-------+
|productID|      date|      customer|rating|votes|helpful|
+---------+----------+--------------+------+-----+-------+
|        2|2002-01-24|A13SG9ACZ9O5IM|     5|    8|      8|
|        2|2002-05-23|A1GIL64QK68WKL|     5|    8|      8|
|        2|2002-02-06|A2P6KAWXJ16234|     4|   16|     16|
|        2|2002-03-23|A3GO7UV9XX14D8|     4|    6|      6|
|        2|2004-02-11|A1CP26N8RHYVVO|     1|   13|      9|
+---------+----------+--------------+------+-----+-------+



### [SQL] - Os 5 comentários mais úteis e com a MENOR avaliação.

In [12]:
spark.sql('SELECT * FROM most_useful_comments ORDER BY rating ASC').show(1000)

+---------+----------+--------------+------+-----+-------+
|productID|      date|      customer|rating|votes|helpful|
+---------+----------+--------------+------+-----+-------+
|        2|2004-02-11|A1CP26N8RHYVVO|     1|   13|      9|
|        2|2002-03-23|A3GO7UV9XX14D8|     4|    6|      6|
|        2|2002-02-06|A2P6KAWXJ16234|     4|   16|     16|
|        2|2002-01-24|A13SG9ACZ9O5IM|     5|    8|      8|
|        2|2002-05-23|A1GIL64QK68WKL|     5|    8|      8|
+---------+----------+--------------+------+-----+-------+



## Usando DataFrame DSL

In [13]:
most_useful_comments_df = reviewsDF.where( reviewsDF.productID == SELECTED_ID ) \
                                .orderBy('helpful', ascending=False) \
                                .limit(5) \
                                .cache()

In [14]:
answer_a_a = most_useful_comments_df.orderBy('rating', ascending=False)
answer_a_b = most_useful_comments_df.orderBy('rating', ascending=True)

### [DSL] - Os 5 comentários mais úteis e com a MAIOR avaliação

In [15]:
answer_a_a.show(1000)

+---------+----------+--------------+------+-----+-------+
|productID|      date|      customer|rating|votes|helpful|
+---------+----------+--------------+------+-----+-------+
|        2|2002-01-24|A13SG9ACZ9O5IM|     5|    8|      8|
|        2|2002-05-23|A1GIL64QK68WKL|     5|    8|      8|
|        2|2002-02-06|A2P6KAWXJ16234|     4|   16|     16|
|        2|2002-03-23|A3GO7UV9XX14D8|     4|    6|      6|
|        2|2004-02-11|A1CP26N8RHYVVO|     1|   13|      9|
+---------+----------+--------------+------+-----+-------+



### [DSL] - Os 5 comentários mais úteis e com a MENOR avaliação

In [16]:
answer_a_b.show(1000)

+---------+----------+--------------+------+-----+-------+
|productID|      date|      customer|rating|votes|helpful|
+---------+----------+--------------+------+-----+-------+
|        2|2004-02-11|A1CP26N8RHYVVO|     1|   13|      9|
|        2|2002-03-23|A3GO7UV9XX14D8|     4|    6|      6|
|        2|2002-02-06|A2P6KAWXJ16234|     4|   16|     16|
|        2|2002-01-24|A13SG9ACZ9O5IM|     5|    8|      8|
|        2|2002-05-23|A1GIL64QK68WKL|     5|    8|      8|
+---------+----------+--------------+------+-----+-------+



---

## Questão B

### Dado um produto, listar os produtos similares com maiores vendas do que ele

### Usando SQL

In [17]:
SELECTED_ID = 2

spark.sql("""
    SELECT * 
        FROM similars s
        INNER JOIN products p ON p.asin = s.asin
        WHERE p.salesrank < (
            SELECT products.salesrank FROM products WHERE products.id = {0}
        )
            AND s.productID = {0}
        ORDER BY p.salesrank

""".format(SELECTED_ID)) \
   .show(1000)

+---------+----------+------+----------+--------------------+-----+---------+-------+------------+------------+
|productID|      asin|    id|      asin|               title|group|salesrank|r_total|r_downloaded|r_avg_rating|
+---------+----------+------+----------+--------------------+-----+---------+-------+------------+------------+
|        2|0738700940|299250|0738700940|              Lammas| Book|    58836|     10|          10|         4.5|
|        2|1567184960| 62291|1567184960|Yule: A Celebrati...| Book|   103012|     35|          35|         3.5|
|        2|0738700525|170507|0738700525|Midsummer: Magica...| Book|   159277|      6|           6|         4.5|
+---------+----------+------+----------+--------------------+-----+---------+-------+------------+------------+



### Usando DataFrame DSL

In [18]:
# Projetar o salesrank do produto selecionado
minimal_salesrank = productsDF.select(productsDF.salesrank) \
                              .where(productsDF.id == SELECTED_ID) \
                              .collect()[0][0]

# Retornar produtos similares ao produto selecionado com um salesrank menor
result = similarsDF.where(similarsDF.productID == SELECTED_ID) \
                   .join(productsDF, productsDF.asin == similarsDF.asin) \
                   .where(productsDF.salesrank < minimal_salesrank) \
                   .orderBy( productsDF.salesrank, ascending=True ) \
                   .show()

+---------+----------+------+----------+--------------------+-----+---------+-------+------------+------------+
|productID|      asin|    id|      asin|               title|group|salesrank|r_total|r_downloaded|r_avg_rating|
+---------+----------+------+----------+--------------------+-----+---------+-------+------------+------------+
|        2|0738700940|299250|0738700940|              Lammas| Book|    58836|     10|          10|         4.5|
|        2|1567184960| 62291|1567184960|Yule: A Celebrati...| Book|   103012|     35|          35|         3.5|
|        2|0738700525|170507|0738700525|Midsummer: Magica...| Book|   159277|      6|           6|         4.5|
+---------+----------+------+----------+--------------------+-----+---------+-------+------------+------------+



---

## Questão C

### Dado um produto, mostrar a evolução diária das médias de avaliação ao longo do intervalo de tempo coberto no arquivo de entrada

### Usando SQL

In [19]:
SELECTED_ID = 19367

answer_c_sql = spark.sql("""
    SELECT 
        r.date as date, 
        r.rating as rating, 
        ( SELECT p.r_avg_rating FROM products p WHERE p.id = {0} ) as avg_rating
    FROM reviews r
    WHERE r.productID = {0}
    ORDER BY r.date
""".format(SELECTED_ID)) \
   .show(1000)

# answer_c_sql.show()
# answer_c = answer_c_sql.rdd.map(tuple).collect()

+----------+------+----------+
|      date|rating|avg_rating|
+----------+------+----------+
|1998-09-21|     5|       4.0|
|1998-10-15|     5|       4.0|
|1998-10-21|     5|       4.0|
|1998-10-23|     2|       4.0|
|1998-11-01|     5|       4.0|
|1998-11-02|     5|       4.0|
|1998-11-02|     5|       4.0|
|1998-11-02|     5|       4.0|
|1998-11-04|     5|       4.0|
|1998-11-10|     2|       4.0|
|1998-11-11|     5|       4.0|
|1998-11-24|     5|       4.0|
|1998-11-27|     2|       4.0|
|1998-12-01|     1|       4.0|
|1998-12-06|     4|       4.0|
|1998-12-11|     5|       4.0|
|1998-12-11|     5|       4.0|
|1998-12-25|     5|       4.0|
|1998-12-26|     5|       4.0|
|1998-12-27|     5|       4.0|
|1998-12-31|     5|       4.0|
|1999-01-10|     5|       4.0|
|1999-01-10|     5|       4.0|
|1999-01-12|     1|       4.0|
|1999-01-17|     5|       4.0|
|1999-01-17|     5|       4.0|
|1999-01-21|     2|       4.0|
|1999-01-23|     3|       4.0|
|1999-01-26|     5|       4.0|
|1999-01

### Usando DataFrame DSL

In [20]:
answer_c = reviewsDF.where( reviewsDF.productID == SELECTED_ID ) \
                    .join( productsDF, reviewsDF.productID == productsDF.id ) \
                    .select( reviewsDF.date, reviewsDF.rating, productsDF.r_avg_rating ) \
                    .show(1000)

+----------+------+------------+
|      date|rating|r_avg_rating|
+----------+------+------------+
|1998-09-21|     5|         4.0|
|1998-10-15|     5|         4.0|
|1998-10-21|     5|         4.0|
|1998-10-23|     2|         4.0|
|1998-11-01|     5|         4.0|
|1998-11-02|     5|         4.0|
|1998-11-02|     5|         4.0|
|1998-11-02|     5|         4.0|
|1998-11-04|     5|         4.0|
|1998-11-10|     2|         4.0|
|1998-11-11|     5|         4.0|
|1998-11-24|     5|         4.0|
|1998-11-27|     2|         4.0|
|1998-12-01|     1|         4.0|
|1998-12-06|     4|         4.0|
|1998-12-11|     5|         4.0|
|1998-12-11|     5|         4.0|
|1998-12-25|     5|         4.0|
|1998-12-26|     5|         4.0|
|1998-12-27|     5|         4.0|
|1998-12-31|     5|         4.0|
|1999-01-10|     5|         4.0|
|1999-01-10|     5|         4.0|
|1999-01-12|     1|         4.0|
|1999-01-17|     5|         4.0|
|1999-01-17|     5|         4.0|
|1999-01-21|     2|         4.0|
|1999-01-2

## Questão D

### Listar os 10 produtos lideres de venda em cada grupo de produto

### Usando SQL

In [30]:
answer_d = spark.sql("""
    SELECT rs.title, rs.group, rs.salesrank
        FROM (
            SELECT title, group, salesrank, ROW_NUMBER() over( Partition BY p.group ORDER BY p.salesrank ASC ) as Rank
                FROM products p
                WHERE salesrank != -1
        ) rs WHERE Rank <= 10
""").show(1000)

+--------------------+------------+---------+
|               title|       group|salesrank|
+--------------------+------------+---------+
|   From Soup to Nuts|       Video|        0|
|The War of the Wo...|       Video|        1|
|   Shirley Valentine|       Video|        2|
|Leslie Sansone - ...|       Video|        6|
|Robin Hood - Men ...|       Video|        7|
|Richard Simmons -...|       Video|        8|
|     Howard the Duck|       Video|       12|
|     Charlotte's Web|       Video|       14|
|A Tree Grows in B...|       Video|       16|
|  My Neighbor Totoro|       Video|       17|
|  IlluStory Book Kit|         Toy|       59|
|Wizard Card Game ...|         Toy|     1890|
|Photostory Junior...|         Toy|     2288|
|Party Tyme Karaok...|         Toy|     4053|
|Party Tyme Karaok...|         Toy|     7812|
|Party Tyme Karaok...|         Toy|    10732|
|The Songs of Brit...|         Toy|    31296|
|R- Photostory Senior|         Toy|    45241|
|         The Drifter|         DVD

### Usando DataFrame DSL

In [32]:
productsDF.where( productsDF.salesrank != -1 ) \
          .withColumn('row_number', F.row_number().over(Window.partitionBy('group').orderBy('salesrank'))) \
          .select('title', 'group', 'salesrank') \
          .where('row_number <= 10') \
          .show(1000)

+--------------------+------------+---------+
|               title|       group|salesrank|
+--------------------+------------+---------+
|   From Soup to Nuts|       Video|        0|
|The War of the Wo...|       Video|        1|
|   Shirley Valentine|       Video|        2|
|Leslie Sansone - ...|       Video|        6|
|Robin Hood - Men ...|       Video|        7|
|Richard Simmons -...|       Video|        8|
|     Howard the Duck|       Video|       12|
|     Charlotte's Web|       Video|       14|
|A Tree Grows in B...|       Video|       16|
|  My Neighbor Totoro|       Video|       17|
|  IlluStory Book Kit|         Toy|       59|
|Wizard Card Game ...|         Toy|     1890|
|Photostory Junior...|         Toy|     2288|
|Party Tyme Karaok...|         Toy|     4053|
|Party Tyme Karaok...|         Toy|     7812|
|Party Tyme Karaok...|         Toy|    10732|
|The Songs of Brit...|         Toy|    31296|
|R- Photostory Senior|         Toy|    45241|
|         The Drifter|         DVD

## Questão E

### Listar os 10 produtos com a maior média de avaliações úteis positivas

### Usando SQL

In [24]:
spark.sql("""
    SELECT p.title, r.productID, AVG(r.helpful) as helpful_mean
        FROM reviews r 
        INNER JOIN products p ON p.id = r.productID
        WHERE r.rating >= 5
        GROUP BY productID, p.title
        ORDER BY helpful_mean DESC
        LIMIT 10
""").show(1000)

+--------------------+---------+------------------+
|               title|productID|      helpful_mean|
+--------------------+---------+------------------+
|Easy Adult Piano ...|   320534|             320.0|
|Small Engine Repa...|    81098|             247.0|
|T'ai Chi for Olde...|   110544|             233.0|
|The Story About Ping|   312621|231.85294117647058|
|The Story About P...|   411581| 231.7058823529412|
|The Story about P...|   403662| 231.6764705882353|
|The Story about Ping|   357193|231.64705882352942|
|The Glucose Revol...|   537547|             206.0|
|More Than Just Ho...|   523037|             203.0|
|Abbott & Costello...|   243584|             197.0|
+--------------------+---------+------------------+



### Usando DataFrame DSL

In [25]:
reviewsDF.where( reviewsDF.rating >= 5 ) \
         .join( productsDF, productsDF.id == reviewsDF.productID ) \
         .select( reviewsDF.productID, productsDF.title, reviewsDF.helpful ) \
         .groupBy( reviewsDF.productID, productsDF.title ) \
         .agg({'helpful':'avg'}) \
         .orderBy('avg(helpful)', ascending=False) \
         .limit(10) \
         .show(1000)

+---------+--------------------+------------------+
|productID|               title|      avg(helpful)|
+---------+--------------------+------------------+
|   320534|Easy Adult Piano ...|             320.0|
|    81098|Small Engine Repa...|             247.0|
|   110544|T'ai Chi for Olde...|             233.0|
|   312621|The Story About Ping|231.85294117647058|
|   411581|The Story About P...| 231.7058823529412|
|   403662|The Story about P...| 231.6764705882353|
|   357193|The Story about Ping|231.64705882352942|
|   537547|The Glucose Revol...|             206.0|
|   523037|More Than Just Ho...|             203.0|
|   243584|Abbott & Costello...|             197.0|
+---------+--------------------+------------------+



## Questão F

### Listar as 5 categorias de produto com a maior média de avaliações úteis positivas por produto

### Usando SQL

In [33]:
spark.sql("""
    SELECT c.category, AVG(r.helpful) as helpful_mean
        FROM reviews r 
        INNER JOIN products p ON p.id = r.productID
        INNER JOIN categories c ON p.id = c.productID
        WHERE r.rating >= 5
        GROUP BY c.category
        ORDER BY helpful_mean DESC
        LIMIT 5
""").show(1000)

+--------------------+----------------+
|            category|    helpful_mean|
+--------------------+----------------+
|Dominican Republi...|97.7872340425532|
|Casual Users[502030]|            84.5|
|Casual Users[727732]|            84.5|
|Jack Benny Progra...|            80.0|
|Adventures of Ozz...|            80.0|
+--------------------+----------------+



### Usando DataFrame DSL

In [34]:
reviewsDF.where( reviewsDF.rating >= 5 ) \
         .join( productsDF, productsDF.id == reviewsDF.productID ) \
         .join( categoriesDF, productsDF.id == categoriesDF.productID ) \
         .select( categoriesDF.category, reviewsDF.helpful ) \
         .groupBy( categoriesDF.category ) \
         .agg({'helpful':'avg'}) \
         .orderBy('avg(helpful)', ascending=False) \
         .limit(5) \
         .show(1000)

+--------------------+----------------+
|            category|    avg(helpful)|
+--------------------+----------------+
|Dominican Republi...|97.7872340425532|
|Casual Users[502030]|            84.5|
|Casual Users[727732]|            84.5|
|Jack Benny Progra...|            80.0|
|Adventures of Ozz...|            80.0|
+--------------------+----------------+



## Questão G

### Listar os 10 clientes que mais fizeram comentários por grupo de produtos

### Usando SQL

In [28]:
answer_d = spark.sql("""
    SELECT rs.group, rs.customer, rs.comments
        FROM (
            SELECT 
                abc.group,
                abc.customer,
                abc.comments,
                ROW_NUMBER() over( Partition BY abc.group ORDER BY abc.comments DESC ) as Rank
            FROM (
                SELECT 
                    p.group, 
                    r.customer, 
                    COUNT(*) as comments
                FROM products p
                INNER JOIN reviews r ON p.id = r.productID 
                GROUP BY p.group, r.customer
            ) abc
        ) rs WHERE Rank <= 10
""").show(1000)

+------------+--------------+--------+
|       group|      customer|comments|
+------------+--------------+--------+
|       Video| ATVPDKIKX0DER|   72581|
|       Video|A3UN6WX5RRO2AG|   15814|
|       Video|A2NJO6YE954DBH|    1775|
|       Video| AU8552YCOO5QX|    1205|
|       Video|A3P1A63Q8L32C5|     737|
|       Video|A20EEWWSFMZ1PN|     720|
|       Video|A16CZRQL23NOIW|     668|
|       Video|A3LZGLA88K0LA0|     614|
|       Video|A2QRB6L1MCJ53G|     606|
|       Video|A152C8GYY25HAH|     583|
|         Toy| AH4M07U4YC695|       2|
|         Toy| ATVPDKIKX0DER|       2|
|         Toy|A1SB7SB31ETYZH|       2|
|         Toy|A1A2RJXP6T26PD|       1|
|         Toy|A2YO9AKVAHDR9I|       1|
|         Toy|A2PJ7WLZ38F47S|       1|
|         Toy|A20AL96IIDAEBU|       1|
|         Toy|A20R67CABJH79P|       1|
|         Toy| AU8PR9XJ17CCB|       1|
|         Toy| AQJ2XVMHXGN9A|       1|
|         DVD| ATVPDKIKX0DER|   63148|
|         DVD|A3UN6WX5RRO2AG|   15549|
|         DVD|A2NJO6YE954

### Usando DataFrame DSL

In [29]:
productsDF.join( reviewsDF, productsDF.id == reviewsDF.productID ) \
          .select( productsDF.group, reviewsDF.customer ) \
          .groupBy( reviewsDF.customer, productsDF.group ) \
          .agg({'customer':'count'}) \
          .withColumn('row_number', F.row_number().over(Window.partitionBy('group').orderBy(desc('count(customer)')))) \
          .where('row_number <= 10') \
          .select( productsDF.group, reviewsDF.customer, 'count(customer)' ) \
          .show(1000)

+------------+--------------+---------------+
|       group|      customer|count(customer)|
+------------+--------------+---------------+
|       Video| ATVPDKIKX0DER|          72581|
|       Video|A3UN6WX5RRO2AG|          15814|
|       Video|A2NJO6YE954DBH|           1775|
|       Video| AU8552YCOO5QX|           1205|
|       Video|A3P1A63Q8L32C5|            737|
|       Video|A20EEWWSFMZ1PN|            720|
|       Video|A16CZRQL23NOIW|            668|
|       Video|A3LZGLA88K0LA0|            614|
|       Video|A2QRB6L1MCJ53G|            606|
|       Video|A152C8GYY25HAH|            583|
|         Toy| AH4M07U4YC695|              2|
|         Toy|A1SB7SB31ETYZH|              2|
|         Toy| ATVPDKIKX0DER|              2|
|         Toy| AQJ2XVMHXGN9A|              1|
|         Toy| AH16IHWEMA61J|              1|
|         Toy|A1LEF9EM2DFDP2|              1|
|         Toy|A1ABBKXKUZF85X|              1|
|         Toy|A1FPVUL053AKXO|              1|
|         Toy|A3O19HBWE10FFJ|     